# Time travel
Using the rank cluster, we can run our collection of real search terms against the new mapping with the new query structure. We can then analyse the results according to the same set of metrics as we used for the data which was collected in real time. In other words, we can look at how search _would have_ performed if we had made these changes earlier. It's a time-travelling A/B test.

## Getting queries
Because the queries are written and tested in typescript, we need to export a json version of them before they can be used in these python notebooks. Running `yarn getQueryJSON <name_of_query_to_test>` will generate a `.json` version of the query alongside the `.ts` original.

We can then import the query as follows:

In [ ]:
import json

In [ ]:
json.loads("data/queries/works-with-search-fields.json")